In [ ]:
import keras, numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, tqdm
import os.path
import time
%matplotlib inline

In [ ]:
import sklearn, sklearn.model_selection

In [ ]:
datadir = os.path.join(os.path.expanduser('~'), 'data2', 'behavioralCloning')

In [ ]:
# data = np.load(os.path.join(datadir, 'multiData-mouseForwardFuller.npz'))
# X = data['X']
# Y = data['Y'][:, 0].reshape(-1, 1)

In [ ]:
import loadData

In [ ]:
# data = loadData.Data(
#     os.path.join(datadir, 'data_provided.zip'), 
#     MAXDATA=1000
# )

In [ ]:
from importlib import reload
reload(loadData)

In [ ]:
data = loadData.Data(
    os.path.join(datadir, 'mouseForward.zip'),
    MAXDATA=2000
)

In [ ]:
# data = loadData.Data(
#     paths=[
#         os.path.join(loadData.HOME, 'data2', 'behavioralCloning', p)
#         for p in (
#             'mouseForward.zip',
#             'mouseReverse.zip',
#         )
#     ],
#     MAXDATA=6000,
# )

In [ ]:
X = data.X
Y = data.Y[:, 0]

In [ ]:
modelname = 'inceptionv3_deeper_wider-mfwd_sidecams-tanh-normalInput'

In [ ]:
X.shape, Y.shape

In [ ]:
def prod(it):
    out = 1
    for x in it:
        try:
            out *= float(x)
        except TypeError:
            out *= int(x)
    return out

In [ ]:
%%time
def VGG(
    nout=1, input_shape=X.shape[1:], doCompile=True, 
    loss='mse', optimizer='nadam', metrics=['accuracy', 'mae'],
    ):
    
    # Normalize.
    x = keras.layers.Input(input_shape)
    x = keras.layers.Lambda(lambda y: (y / 255.0) - 0.5)(x)
    
    model = keras.applications.inception_v3.InceptionV3(
        include_top=False, 
        weights='imagenet', 
        input_tensor=x,
        #input_shape=input_shape, 
        #pooling=None, 
        #classes=1000
    )
    img_input = model.layers[0].input
    nUnfrozen = sum([prod(w.shape) for w in model.trainable_weights])
    for layer in model.layers:
        layer.trainable = False
    
    # Get flattened output.
    x = model.layers[-1].output
    print('Pretrained model provides {:,} features.'.format(prod(x.shape[1:])))
    x = keras.layers.Flatten()(x)
    
    
    # Add layers.
    init = dict(bias_initializer='zeros', kernel_initializer='glorot_normal')
    x = keras.layers.Dense(16, activation='relu', **init)(x)
#     for k in range(3):
#         x = keras.layers.Dense(16, activation='relu', **init)(x)
#     x = keras.layers.Dense(16, activation='tanh', name='tanh')(x)
#     x = keras.layers.Dense(nout, activation='linear', name='predictions')(x)
    for k in range(4):
        x = keras.layers.Dense(16, activation='relu', **init)(x)
    x = keras.layers.Dense(nout, activation='tanh', name='tanh')(x)
    
    model = keras.Model(inputs=img_input, outputs=x)
    
    nFrozen = sum([prod(w.shape) for w in model.trainable_weights])
    print("Freezing reduces trainable size from {:,} to {:,} parameters.".format(nUnfrozen, nFrozen))
    
    if doCompile:
        model.compile(
            loss=loss,
            optimizer=optimizer,
            metrics=metrics,
        )
    return model

model = VGG(
    optimizer=keras.optimizers.Nadam(
        lr=0.0001, #beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    )
)


In [ ]:
class TqdmCallback(keras.callbacks.Callback):
    def __init__(self, nbatch):
        self.pbar = tqdm.tqdm_notebook(total=nbatch, unit='epoch')
    
    def on_train_begin(self, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        self.pbar.update()

In [ ]:
NEPOCH = 32

In [ ]:
log_dir = '/home/tsbertalan/tensorboardlogs/behavClon/%s-%s/'% (modelname, time.time())

In [ ]:
%%time
history = model.fit(
    X, Y,
    validation_split=.2,
    #X_train, Y_train, 
    #validation_data=(X_valid, Y_valid), 
    epochs=NEPOCH,
    callbacks=[
        TqdmCallback(NEPOCH),
        keras.callbacks.TensorBoard(
            log_dir=log_dir,
            #batch_size=32, 
            write_graph=True, 
        ),
    ],
    verbose=0
)

In [ ]:
%%time
fpath = os.path.join(datadir, '%s.h5' % modelname)
print('Saving to', fpath)
model.save(fpath)

In [ ]:
!recent /home/tsbertalan/data2/behavioralCloning/ | head -n 4

In [ ]:
pred = model.predict(X)
pred.shape

In [ ]:
def runningMeanFast(x, N):
    return np.convolve(x.ravel(), np.ones((N,))/N)[(N-1):]

In [ ]:
fig, ax = plt.subplots()
start = 1450
end = start + 200
ax.plot(Y[start:end], label=r'truth $\theta$')
ax.plot(pred[start:end], label=r'predictions $\hat\theta$')
filtersize = 4
scale = 1
ax.plot(
    scale * runningMeanFast(pred[start:end], filtersize), 
    label=r'$\hat\rho = %.1f \cdot box_{%d}(\hat\theta)$' % (scale, filtersize)
)
ax.legend();